In [7]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import threading
import webbrowser
from dash.dependencies import Input, Output, State

In [8]:
file_path = "../Spotify_top100/spotify_top100.csv"

In [9]:
df = pd.read_csv(file_path)

In [10]:
df.head(5)

,title,artist,top genre,year released,added,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top year,artist type
0,STARSTRUKK (feat. Katy Perry),3OH!3,dance pop,2009.0,2022‑02‑17,140.0,81.0,61.0,-6.0,23.0,23.0,203.0,0.0,6.0,70.0,2010.0,Duo
1,My First Kiss (feat. Ke$ha),3OH!3,dance pop,2010.0,2022‑02‑17,138.0,89.0,68.0,-4.0,36.0,83.0,192.0,1.0,8.0,68.0,2010.0,Duo
2,I Need A Dollar,Aloe Blacc,pop soul,2010.0,2022‑02‑17,95.0,48.0,84.0,-7.0,9.0,96.0,243.0,20.0,3.0,72.0,2010.0,Solo
3,Airplanes (feat. Hayley Williams of Paramore),B.o.B,atl hip hop,2010.0,2022‑02‑17,93.0,87.0,66.0,-4.0,4.0,38.0,180.0,11.0,12.0,80.0,2010.0,Solo
4,Nothin' on You (feat. Bruno Mars),B.o.B,atl hip hop,2010.0,2022‑02‑17,104.0,85.0,69.0,-6.0,9.0,74.0,268.0,39.0,5.0,79.0,2010.0,Solo


In [12]:
example_df = df[df['top year'] == 2010].nlargest(15, 'bpm')
example_df = example_df.sort_values(by='bpm', ascending=True)


In [13]:
fig = px.bar(
    example_df,
    y='title',
    x='bpm',
    text='bpm',
    color_discrete_sequence=["#1DB954"],
    labels={'bpm': 'Tempo (BPM)', 'title': 'Song Title'},
    title=f"Top 15 Songs by Tempo",
    template='plotly_dark',
)

fig.update_layout(
    height=600,
    xaxis_title="Beats per Minute (BPM)",
    yaxis_title="Song Title",
    title_font_size=20,
    title_x=0.5,
    margin=dict(l=40, r=40, t=50, b=40),
)
fig.update_traces(textposition='outside')
fig.show()

In [14]:
app = dash.Dash(__name__)

# App layout
app.layout = html.Div(children=[
    # Title
    html.H1(
        "Spotify Top 100 Ranking",
        style={'textAlign': 'center', 'marginBottom': '20px', 'color': '#1DB954'}
    ),
    
    # Slider title
    html.Label(
        "Select Year:",
        style={'fontSize': '18px', 'marginBottom': '10px', 'display': 'block', 'color': '#ffffff'}
    ),
    
    # Slider
    dcc.Slider(
        id='year-slider',
        min=df['top year'].min(),
        max=df['top year'].max(),
        step=1,
        marks={year: str(year) for year in sorted(df['top year'].unique())},
        value=df['top year'].min()
    ),
    
    # Dropdown title
    html.Label(
        "Select Parameter:",
        style={'fontSize': '18px', 'marginBottom': '10px', 'display': 'block', 'color': '#ffffff'}
    ),
    
    # Dropdown for parameter selection
    dcc.Dropdown(
        id='parameter-dropdown',
        options=[
            {'label': 'Tempo (BPM)', 'value': 'bpm'},
            {'label': 'Song Energy', 'value': 'nrgy'},
            {'label': 'Danceability', 'value': 'dnce'},
            {'label': 'Volume (dB)', 'value': 'dB'},
            {'label': 'Live Recording Probability', 'value': 'live'},
        ],
        value='bpm',
        style={'fontSize': '16px', 'color': '#191414'}
    ),
    
    # Graph
    dcc.Graph(
        id='ranking-chart',
        style={'height': '600px'}
    )
], style={'padding': '20px', 'fontFamily': 'Arial, sans-serif', 'backgroundColor': '#191414'})

@app.callback(
    Output('ranking-chart', 'figure'),
    [Input('year-slider', 'value'),
     Input('parameter-dropdown', 'value')]
)
def update_chart(selected_year, selected_parameter):
    filtered_df = df[df['top year'] == selected_year]
    filtered_df = filtered_df.nlargest(15, selected_parameter)
    filtered_df = filtered_df.sort_values(by=selected_parameter, ascending=True)

    # Creating a bar chart
    fig = px.bar(
        filtered_df,
        x=selected_parameter,
        y='title',
        text=selected_parameter,
        color_discrete_sequence=["#1DB954"],
        labels={selected_parameter: selected_parameter.capitalize(), 'title': 'Song Title'},
        title=f"Top 15 Songs by {selected_parameter.capitalize()} in {selected_year}",
        template='plotly_dark'
    )

    # Update the chart layout
    fig.update_layout(
        height=600,
        xaxis_title=f"{selected_parameter.capitalize()} Value",
        yaxis_title="Song Title",
        title_font_size=20,
        title_x=0.5,
        margin=dict(l=40, r=40, t=50, b=40),
    )
    fig.update_traces(textposition='outside')

    return fig

# Run app
def run_app():
    app.run_server(debug=False, use_reloader=False)

# Starting the app in a new thread and opening it in the browser
thread = threading.Thread(target=run_app)
thread.start()

# Open the application in the browser
webbrowser.open("http://127.0.0.1:8050/")

True